In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Embedding, Activation, Flatten
from keras.optimizers import Adam
import numpy as np
import re
import preprocessing as preproc
import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import helpers
import models

Using TensorFlow backend.


In [2]:
start_time = time.time()

In [ ]:
preproc.do_preprocessing('../data/twitter-datasets/train_pos.txt')
preproc.do_preprocessing('../data/twitter-datasets/train_neg.txt')

In [58]:
lines, y = preproc.return_processed_trainset_and_y(False)
len(lines)

200000

In [ ]:
lines1 = lines[:10000] + lines[100000:110000]
y1 = np.concatenate((y[:10000], y[100000:110000]), axis=0)
y1 = y1.tolist()
print(len(lines1))
print(len(y1))

In [59]:
np.unique(y)

array([0., 1.])

In [99]:
vectorizer = TfidfVectorizer(stop_words=None, ngram_range=(1,2), sublinear_tf=True, max_features=1000)
# X = vectorizer.fit(lines)
# X = X.todense()
# X = X.transform(lines)
X = vectorizer.fit_transform(lines)

In [100]:
type(X)

scipy.sparse.csr.csr_matrix

In [113]:
adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.001, amsgrad=False)

In [114]:
# create model
model = Sequential()
# model.add(Embedding(X.shape[0], 300, input_length=X.shape[1]))
# model.add(Flatten())
model.add(Dense(units=64, input_dim=X.shape[1], activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 64)                64064     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_41 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 33        
Total params: 66,177
Trainable params: 66,177
Non-trainable params: 0
_________________________________________________________________


In [115]:
#compile model
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['acc'])

In [ ]:
#fit the model
model.fit(x=X, y=y, batch_size=32, epochs=100, verbose=1)

Epoch 1/100
200000/200000 [==============================] - 27s 134us/step - loss: 0.4521 - acc: 0.7777
Epoch 2/100
200000/200000 [==============================] - 26s 129us/step - loss: 0.4212 - acc: 0.7977
Epoch 3/100
200000/200000 [==============================] - 26s 130us/step - loss: 0.4093 - acc: 0.8049
Epoch 4/100
200000/200000 [==============================] - 27s 133us/step - loss: 0.4012 - acc: 0.8098
Epoch 5/100
200000/200000 [==============================] - 28s 138us/step - loss: 0.3955 - acc: 0.8135
Epoch 6/100
200000/200000 [==============================] - 27s 133us/step - loss: 0.3912 - acc: 0.8163
Epoch 7/100
200000/200000 [==============================] - 26s 132us/step - loss: 0.3886 - acc: 0.8173
Epoch 8/100
200000/200000 [==============================] - 26s 130us/step - loss: 0.3857 - acc: 0.8198
Epoch 9/100
200000/200000 [==============================] - 26s 131us/step - loss: 0.3833 - acc: 0.8211
Epoch 10/100
200000/200000 [===========================

In [ ]:
scores = model.evaluate(X, y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
model.fit_generator(generator=batch_generator(X, y1, 32),
                   epochs=5, validation_data=(X,y1),
                    steps_per_epoch=X.shape[0]/32)

In [ ]:
def batch_generator(X, y, batch_size):
    samples_per_epoch = X.shape[0]
    num_of_batches = samples_per_epoch / batch_size
    counter = 0
    index = np.arange(np.shape(y)[0])
    while True:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[index_batch,:].toarray()
        y_batch = y[y.index[index_batch]]
        counter += 1
        yield X_batch, y_batch
        if counter > num_of_batches:
            counter = 0

400 features
Me 10000 data:
# 78.35 Adam:decay 0.002 lr 0.05 epochs 100
# 73 Adam:decay 0.02

Me 20000 data:
# 77.23 Adam:decay 0.002 lr 0.05 epochs 100

Me 200000 data:
# 78.22 Adam:decay 0.002 lr 0.05 epochs 100

Ola

 78.94% adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.001, amsgrad=False)
 
 79.40% adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.0001, amsgrad=False)
 
Features 1000

 81% adam = Adam(lr=0.03, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.0001, amsgrad=False)
  
 83 % adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.0001, amsgrad=False)